In [2]:
import csv
import matplotlib.pyplot as plt  
import pandas as pd
import numpy as np
import sys

Implement full version of linear regression manually
====================================================

In [3]:
# Step 1: Load
X = []  # will hold [1, feature1, feature2]
y = []

feature1_sum = 0
feature2_sum = 0
target_sum = 0

with open('../data/multiple_linear_regression_dataset.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        X.append([1.0, float(row['age']), float(row['experience'])])
        y.append(float(row['income']))
        print(f"Loaded row: {row}")

        feature1_sum += float(row['age'])
        feature2_sum += float(row['experience'])
        target_sum += float(row['income'])

m = len(y)  # number of samples
feature1_mean = feature1_sum / m
feature2_mean = feature2_sum / m
target_mean = target_sum / m
print(f"Means: feature1={feature1_mean}, feature2={feature2_mean}, target={target_mean}")

# Step 2 & 3: Closed-form solution (manual matrix math)
# Compute X^T X -> 3x3 matrix, X^T y -> 3-vector
X_T = [[X[j][i] for j in range(m)] for i in range(3)]  # Transpose X
X_T_X = [[sum(X_T[i][k] * X_T[j][k] for k in range(m)) for j in range(3)] for i in range(3)]
X_T_y = [sum(X_T[i][j] * y[j] for j in range(m)) for i in range(3)]

# Then compute inverse of 3x3 manually (via formula)
def invert_matrix(matrix):
    a, b, c = matrix[0]
    d, e, f = matrix[1]
    g, h, i = matrix[2]

    det = a * ((e * i) - (f * h)) - b * ((d * i) - (f * g)) + c * ((d * h) - (e * g))

    if (det == 0):
        sys.exit("Determinate is zero. Cannot invert matrix.")
    else: 
        sub_matrix = [[0 for _ in range(2)] for _ in range(2)]
        M = [[0 for _ in range(3)] for _ in range(3)]

        for i in range(3):
            for x in range(3):
                sub_matrix = [[0, 0], [0, 0]]
                k = 0
                for j in range(3):
                    if (j == i):
                        continue
                    z = 0
                    for y in range(3):
                        if (y == x) :
                            continue
                        sub_matrix[k][z] = matrix[j][y]
                        z += 1
                    k += 1
                    
                minor = (sub_matrix[0][0] * sub_matrix[1][1]) - (sub_matrix[0][1] * sub_matrix[1][0])
                cofactor = (-1) ** (i + x)
                M[i][x] = (minor * cofactor) / det
        
        M_T = [[M[j][i] for j in range(3)] for i in range(3)]

    return M_T

X_T_X_inv = invert_matrix(X_T_X)

# Multiply inverse by X^T y to get beta vector
beta = [sum(X_T_X_inv[i][j] * X_T_y[j] for j in range(3)) for i in range(3)]
print(f"Beta: {beta}")
# Step 4: Predictions & MSE
predictions = [sum(beta[j] * X[i][j] for j in range(3)) for i in range(m)]
mse_closed = sum((pred - actual)**2 for pred,actual in zip(predictions, y)) / m
rmse = mse_closed ** 0.5
print(f"RMSE: {rmse}")

# Step 6: Gradient Descent
# Initialize theta = [0, 0, 0]
# For each iteration:
#   predictions = [...]
#   gradient = one vector of size 3 = (1/m) * X^T (preds - y)
#   theta = [theta_j - alpha * grad_j for each j]
#   compute cost and store

# Finally compare

Loaded row: {'age': '25', 'experience': '1', 'income': '30450'}
Loaded row: {'age': '30', 'experience': '3', 'income': '35670'}
Loaded row: {'age': '47', 'experience': '2', 'income': '31580'}
Loaded row: {'age': '32', 'experience': '5', 'income': '40130'}
Loaded row: {'age': '43', 'experience': '10', 'income': '47830'}
Loaded row: {'age': '51', 'experience': '7', 'income': '41630'}
Loaded row: {'age': '28', 'experience': '5', 'income': '41340'}
Loaded row: {'age': '33', 'experience': '4', 'income': '37650'}
Loaded row: {'age': '37', 'experience': '5', 'income': '40250'}
Loaded row: {'age': '39', 'experience': '8', 'income': '45150'}
Loaded row: {'age': '29', 'experience': '1', 'income': '27840'}
Loaded row: {'age': '47', 'experience': '9', 'income': '46110'}
Loaded row: {'age': '54', 'experience': '5', 'income': '36720'}
Loaded row: {'age': '51', 'experience': '4', 'income': '34800'}
Loaded row: {'age': '44', 'experience': '12', 'income': '51300'}
Loaded row: {'age': '41', 'experience'

Implementation with numpy and pandas
====================================

In [4]:
data = pd.read_csv('../data/multiple_linear_regression_dataset.csv')
df = pd.DataFrame(data)

df.describe()

,age,experience,income
count,20.000000,20.000000,20.000000
mean,39.650000,6.200000,40735.500000
std,10.027725,4.124382,8439.797625
min,23.000000,1.000000,27840.000000
25%,31.500000,3.750000,35452.500000
50%,40.000000,5.000000,40190.000000
75%,47.000000,9.000000,45390.000000
max,58.000000,17.000000,63600.000000


Closed-form solution (manual matrix math)
=========================================

In [5]:
# Closed-form solution (manual matrix math)
print("Closed-form solution (manual matrix math)")
# Compute X^T X -> 3x3 matrix, X^T y -> 3-vector
y = df['income'].to_numpy()
X = df[['age', 'experience']].copy()
X['intercept'] = 1
X = X.to_numpy()

X_T = X.T
X_T_X = X_T @ X
X_T_y = X_T @ y

# Then compute inverse of 3x3 manually (via formula)
invert_matrix = np.linalg.inv(X_T_X)

# Multiply inverse by X^T y to get beta vector
B = np.dot(invert_matrix, X_T_y)
print(f"Beta: {B}")

# Predictions & MSE
predictions = X @ B
mse_closed = sum((pred - actual)**2 for pred, actual in zip(predictions, y)) / len(y)

rmse = np.sqrt(mse_closed)
print(f"RMSE: {rmse}")

Closed-form solution (manual matrix math)
Beta: [  -99.19535546  2162.40419192 31261.6898541 ]
RMSE: 1238.3997653077054


Gradient descent solution
=========================

In [6]:
# Gradient descent solution
print("Gradient descent solution")

y = df['income'].to_numpy()
X = df[['age', 'experience']].copy()
X['intercept'] = 1
X = X.to_numpy()

a = 0.0001
beta = np.zeros(3)
n = len(y)

rmse = 2001
while(rmse > 1240):
    predictions = X @ beta
    err = predictions - y
    gradient = (2 / n) * (X.T @ err)
    beta -= a * gradient

    mse = np.mean(err ** 2)
    rmse = np.sqrt(mse)

print(f"Beta: {beta}")
print(f"RMSE: {rmse}")

Gradient descent solution
Beta: [  -91.78284612  2156.84784506 30987.77141271]
RMSE: 1239.9999782625264


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y = df['income'].to_numpy()
X = df[['age', 'experience']].copy()
X['intercept'] = 1
X = X.to_numpy()

model = LinearRegression()
model.fit(X, y)

preds = model.predict(X)

mse = mean_squared_error(y, preds)
rmse = np.sqrt(mse)

beta = model.coef_

print(f"Beta: {beta}")
print(f"RMSE: {rmse}")

Beta: [ -99.19535546 2162.40419192    0.        ]
RMSE: 1238.3997653077047
